In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
train = pd.read_csv('train_Winterpolate_stn+ef_label.csv')
weighted_df = train.copy()

In [6]:
weighted_df = weighted_df.drop(columns=['ef_datetime'])
weighted_df.head()

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,vv,class_interval
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,5,1,12,0.0,0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,5,1,15,0.0,0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,5,1,18,0.0,0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,5,1,21,0.0,0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,5,2,0,0.0,0


In [16]:
from pycaret.regression import *  
regression_setup = setup(
    data=weighted_df.drop(columns=['class_interval']), 
    target='vv',  # 목표 변수 설정
    numeric_features=['v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09'],  # 수치형 특징 설정
    categorical_features=['stn4contest', 'year', 'month', 'day', 'hour'],  # 범주형 특징 설정
    session_id=123,  # 재현성을 위한 세션 ID 설정
    use_gpu=True,  # GPU 사용
    fold_strategy='kfold',  # K-Fold 교차 검증 사용
    fold=10,  # Fold 수 설정
    verbose=False  # 출력을 최소화
)

In [17]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7787,11.2185,3.3279,0.4036,0.4234,2.6928,1.8350
rf,Random Forest Regressor,0.8118,11.5345,3.3746,0.3868,0.4512,3.0462,38.4970
br,Bayesian Ridge,0.9296,11.7564,3.4083,0.3741,0.4909,3.0846,1.5760
lr,Linear Regression,0.9381,11.7577,3.4086,0.3739,0.4937,3.0940,1.5830
ridge,Ridge Regression,0.9380,11.7577,3.4086,0.3739,0.4937,3.0939,1.4100
en,Elastic Net,0.8935,11.9894,3.4378,0.3647,0.4811,3.0501,1.5720
gbr,Gradient Boosting Regressor,0.8014,12.0587,3.4506,0.3583,0.4304,2.7166,49.4070
llar,Lasso Least Angle Regression,0.8924,12.1440,3.4583,0.3578,0.4809,3.0471,1.3830
lasso,Lasso Regression,0.8923,12.1440,3.4583,0.3578,0.4809,3.0465,1.6750
omp,Orthogonal Matching Pursuit,0.8996,12.1337,3.4584,0.3572,0.4820,3.0697,1.4270


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [18]:
tuned_model = tune_model(best_model, n_iter=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8597,18.7205,4.3267,0.3158,0.4416,3.0362
1,0.7614,8.2977,2.8806,0.3912,0.4334,2.7016
2,0.7912,9.6186,3.1014,0.4052,0.4338,2.4722
3,0.8042,14.0130,3.7434,0.3796,0.4210,2.7828
4,0.7541,9.8992,3.1463,0.4259,0.4128,2.5891
5,0.7149,9.0465,3.0077,0.3757,0.4107,2.8434
6,0.7931,13.0901,3.6180,0.3970,0.4171,2.7001
7,0.7448,8.8695,2.9782,0.4321,0.4269,2.2560
8,0.8412,13.0598,3.6138,0.4160,0.4319,2.5593


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 5 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [19]:
save_model(tuned_model, 'best_model_winter')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['v01', 'v02', 'v03', 'v04', 'v05',
                                              'v06', 'v07', 'v08', 'v09'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['stn4contest', 'year', 'month',
                                              'day', 'hour'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['stn4contest', 'year', 'month',
                                              'hour'],
                                     transformer=OneHotEncoder(cols=['stn4contest',
                                                                     'year',
                                                                     'month',
               

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

test = pd.read_csv('test_stn+ef_label.csv')
submit = pd.read_csv('rainfall_test.csv')

prediction_test = predict_model(tuned_model,  data=test)
idx999 = prediction_test[prediction_test['class_interval'].notnull()].index
print(idx999)

NameError: name 'predict_model' is not defined

In [4]:
test = pd.read_csv('test_stn+ef_label.csv')
test.head()

,stn4contest,ef_datetime,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,class_interval
0,31,2023-05-01 12:00:00,68.0,46.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,12,NaN
1,31,2023-05-01 15:00:00,83.0,66.0,26.0,5.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,15,NaN
2,31,2023-05-01 18:00:00,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,18,NaN
3,31,2023-05-01 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,21,NaN
4,31,2023-05-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,2,0,NaN


In [22]:
def classify_class(rainfall):
    if rainfall < 0.1:
        return 0
    elif 0.1 <= rainfall < 0.2:
        return 1
    elif 0.2 <= rainfall < 0.5:
        return 2
    elif 0.5 <= rainfall < 1.0:
        return 3
    elif 1.0 <= rainfall < 2.0:
        return 4
    elif 2.0 <= rainfall < 5.0:
        return 5
    elif 5.0 <= rainfall < 10.0:
        return 6
    elif 10.0 <= rainfall < 20.0:
        return 7
    elif 20.0 <= rainfall < 30.0:
        return 8
    else:
        return 9


In [23]:
#prediction_train['prediction_cl'] = prediction_train['prediction_label'].apply(classify_class)
prediction_test['class_interval'] = prediction_test['prediction_label'].apply(classify_class)
prediction_test.loc[idx999, 'class_interval'] = -999

#############--------------------------------------------------
#rainfall_test. 제거 데이터
submit=pd.read_csv("rainfall_test2.csv")
#제출용 데이터
realsubmit=pd.read_csv("rainfall_test.csv")


#submit-prediction 지점,date 맞추기
prediction_test['ef_datetime'] = pd.to_datetime(prediction_test['ef_datetime'])
submit['ef_year_temp'] = submit['ef_year'].replace({'D': '2023'})
submit['ef_datetime'] = pd.to_datetime(submit[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), 
                                       format='%Y-%m-%d-%H')
stn_mapping = {'STN031': 31, 'STN032': 32, 'STN033': 33, 'STN034': 34, 'STN035': 35}
submit['stn4contest'] = submit['stn4contest'].replace(stn_mapping)



#prediction의 class_interval submit에 흩뿌리기
# prediction_test=new_data.copy() #new_data : 개개인마다 달라유
merged = submit.merge(prediction_test[['ef_datetime', 'stn4contest', 'class_interval']], 
                      on=['ef_datetime', 'stn4contest'], 
                      how='left', 
                      suffixes=('', '_pred'))
merged['class_interval_pred']
#-999 index 저장
idx999 = realsubmit[realsubmit['rainfall_test.class_interval'].notnull()].index
merged.loc[idx999, 'class_interval_pred'] = -999
#제출용 데이터에 class_interval 채우기
realsubmit['rainfall_test.class_interval']=merged['class_interval_pred']
#저장
realsubmit.to_csv("240494.csv")  # 각자 접수번호.csv 로 저장

In [24]:
realsubmit['rainfall_test.class_interval'].value_counts()

rainfall_test.class_interval
 0      78867
 2       8726
 5       8279
 4       6561
 3       6339
 1       5619
 6       4576
 7       2144
 8        559
 9        200
-999      130
Name: count, dtype: int64